In [ ]:
# IGNORE THIS CELL WHICH CUSTOMIZES LAYOUT AND STYLING OF THE NOTEBOOK !
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import warnings

import matplotlib.pyplot as plt

warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings = lambda *a, **kw: None
from IPython.core.display import HTML

HTML(open("../documents/custom.html", "r").read())

<br/>
<span style="background:#f0f0e0;padding:1em">Copyright (c) 2020-2021 ETH Zurich, Scientific IT Services. This work is licensed under <a href="https://creativecommons.org/licenses/by-nc/4.0/">CC BY-NC 4.0</a></span><br/>
<br/>

<p style="font-size: 2.5em; font-weight: bold;">Section 5: Parallel computing on a PC</p> 

In modern scientific computing **parallel computing** is omnipresent.

Nowadays, it is common to have multicore processors even on a laptop. This opens up opportunities to solve some of the computational problems faster by leveraging these parallel architectures. 

Solving a problem means identifying the steps (or set of instructions) that allow us to solve it, i.e. the **algorithm**. If all the steps are executed in a non-parallel fashion, then the algorithm is said to be **serial**.

In simple terms, **parallel computing** refers to breaking down of the computational task into smaller "independent" chunks and executing these chunks concurrently (may or may not be truly parallel) or in-parallel using different mechanisms. In practice, we might have to reconsider the initial **serial** algorithm for one that allows better parallelization.

# Thinking parallel

Before embarking on a journey to start parallelizing our code, we first need to think if the problem at hand is parallelizable at all. 

For example, an algorithm in which the computation at any given step depends on the previous steps cannot be parallelized.

<div class="alert alert-block alert-info">
    <p style="font-weight: bold;"><i class="fa fa-info-circle"></i> A typical serial problem: Fibonacci series (0,1,1,2,3,5,8...)</p>
    
$F(n) = F(n-1) + F(n-2)$
    
In the formula stated above the function $F$ at step $n$ depends on its value at steps $n-1$ and $n-2$, respectively. Therefore, we need to compute these values before we can go ahead with the computation at step $n$. This procedure does not lends itself to parallelization.
</div>


If the problem is parallelizable, then there are two prominent ways we can approach it:

* **Domain decomposition**

In this approach, one aims to partition the data which needs to be processed such that the tasks/instructions can operate on separate chunks of data, simultaneously.

<p style="text-align:center;">
    <img src="./images/domain_decomp.gif" width="300" alt="domain"/>
    <div style="text-align:center;">Source: <a href="https://hpc.llnl.gov/training/tutorials/introduction-parallel-computing-tutorial">LLNL Introduction to Parallel Computing Tutorial</a></div>
</p>


* **Functional decomposition**

In this approach, the computation task itself is partitioned such that different tasks do parts of the overall work.

<p  style="text-align:center;">
    <img src="./images/functional_decomp.gif" width="400" alt="domain"/>
    <div style="text-align:center;">Source: <a href="https://hpc.llnl.gov/training/tutorials/introduction-parallel-computing-tutorial">LLNL Introduction to Parallel Computing Tutorial</div></a>
</p>

# Scaling: Can I just throw in infinite CPUs to get infinite speedup?

**Short answer**: of course, NO!

**Detailed answer**

In order to give a detailed answer, we first need to set the stage.

Let's assume that we have a problem that can be parallelized. Following that we assume that the problem depends linearly on its input size $N$ (called also **problem size**) and can executed on $P$ processors. 

Let's say we can solve the problem using the **fastest serial** algorithm in time $T_S$. 

We can also use a parallel algorithm on $P$ processor and solve the same problem in time $T(P,N)$. In other words, problem of size $N$ is solved on $P$ processors in time $T(P,N)$. 

It is important to notice that generally when using the parallel algorithm on $P=1$ processors the problem is solved in $T(1,N) \ge T_S$. 

The parallel algorithm usually contains a fraction $f$ that is inherently serial and cannot be parallelized.

To quantify the benefit obtained from parallelization, it is useful to define some metrics:

<div class="alert alert-block alert-warning">

<div style="font-size: 150%;"></div>
<i class=\"fa fa-info-circle\"></i>

**Absolute Speedup**: 
    
    
Ratio of the time take by the **best** serial algorithm to solve a particular problem ($T_S$) to the parallel execution time on $P$ processors ($T(P,N)$).
    
    
$$S^{abs}(P,N) :=\frac{\textrm{time taken by the best serial algorithm}}{\textrm{time taken by the parallel algorithm on } P \textrm{ processors}} = \frac{T_S}{T(P,N)} .$$
    
    
    
**Relative Speedup**: 
    
Ratio of serial time to the parallel execution time on $P$ processors using the parallel algorithm
$$S(P,N) := \frac{T(1,N)}{T(P,N)} \ge \frac{T_S}{T(P,N)} = S^{abs}(P,N).$$
    
    
    
**Relative Efficiency**:
    
Ratio of the speedup to the number of processors used. Measures the fraction of the time for which each processor is usefully utilized
$$E(P,N) := \frac{S(P,N)}{P} .$$
    
    
    
**Scalability**:
    
It is a measure of a parallel algorithm's increase in speedup with an increasing number of processors

</div>

There are two type of **scalabilities** (strong and weak) which you will often see in literature.

## Strong scaling - Amdahl's law

Strong scaling concerns the speed up achieved by increasing the number of processors ($P$) while the **problem size ($N$) remains fixed**.

The speedup with respect to strong scaling is expressed by the so-called **Amdahl's law**. 

Let's say an algorithm contains a fraction $f$ of work which is serial (not parallelizable) (Therefore $1-f$ is the fraction of work that is parallelizable).

According to Amdahl's law, the theoretical speedup obtained by running this algorithm on $P$ processors is given by:

$$S(P,N) = \frac{1}{f + \frac{1-f}{P}}$$

<div class="alert alert-block alert-info">
    <p style="font-weight: bold;"><i class="fa fa-info-circle"></i>Derivation:</p>

As stated above, the execution time of a parallel algorithm on a single processor is $T(1,N)$. Let's assume an algorithm, fraction $f$ of which is non parallelizable.
We can express the time taken by the parallel code using $P$ processors is:
    
$$T(P,N)=f\times T(1,N) + (1-f)\times \frac{T(1,N)}{P} = T(1,N) \times \left( f + \frac{1-f}{P}\right) .$$

Then the theoretical speedup of this algorithm is:

$$S(P,N) = \frac{T(1,N)}{T(P,N)}=\frac{T(1,N)}{T(1,N) \times \left( f + \frac{1-f}{P}\right)} = \frac{1}{f + \frac{1-f}{P}} \le \frac{1}{f} .$$
</div>

We can notice that the problem can be solved $P$ times faster on $P$ processors only when $f=0$ (i.e. there is no serial part). When $f$ is non-zero the maximum theoretical speedup which corresponds to an infinite number of processors is
$$S_{max}=\frac{1}{f}$$ because  $\frac{1-f}{p} \to 0$ and therefore it is given by the serial fraction.

To demonstrate the importance of this fact, let's say that we are very lucky and only 10% of our code is non-parallelizable and the rest 90% is.  
Let's plot the speedup as a function of number of processors we throw in to solve the problem

In [ ]:
import matplotlib.pyplot as plt
import numpy as np


def amdahl_speedup(f, P):
    speedup = 1 / (f + ((1 - f) / P))
    return speedup


def speedup_plot(speedup, f, P, title):
    max_speedup = round(1 / f, 2)
    plt.plot(P, speedup, label="f={}, S_max ={}".format(f, max_speedup))
    # Add title and axis names
    plt.title(title)
    plt.xlabel("Number of processors")
    plt.ylabel("Speedup")
    # Limits for the Y axis
    plt.xlim(0, num_processors + 10)
    plt.grid(True)
    plt.legend()
    plt.tight_layout()


num_processors = 200
P = np.arange(1, num_processors, 1, dtype=int)
f = [0.1]
for fi in f:
    speedup = amdahl_speedup(fi, P)
    speedup_plot(speedup, fi, P, "Amdahl's law")

Now let's plot the speedup as a function of the serial fraction of the code

In [ ]:
num_processors = 200
P = np.arange(1, num_processors, 1, dtype=int)
f = [0.1, 0.25, 0.33, 0.5]
for fi in f:
    speedup = amdahl_speedup(fi, P)
    speedup_plot(speedup, fi, P, "Amdahl's law")

From the plots above we can see that the maximum speedup obtainable for an algorithm to solve a problem of fixed size is strictly limited by its serial fraction.
<div class="alert alert-block alert-warning">

<div style="font-size: 150%;"></div>
<i class=\"fa fa-info-circle\"></i>

**Take home message:** we are limited in the speedup by the serial fraction and we cannot efficiently use more and more processors. As stated above, the maximum speedup obtainable is give by

$$S_{max}=\frac{1}{f}$$
</div>

Question to audience:
**Why do we need big clusters and supercomputers?!**

## Weak scaling - Gustafson's law

In this scaling analysis, the problem size ($N$) is increased proportional to the number of processors added ($P$).

The speedup with respect to weak scaling is expressed by the so-called **Gustafson's law**. 

Let's consider that the parallel work performed varies linearly with the increased number of processors such that the **execution time stays constant** and that in such a case the **serial component is constant** and is therefore is independent of the problem size.

Let's say a parallel algorithm takes time $T_{serial}$ for the serial part and  $T_{parallel}$ for the parallel part on $P$ processors:
$$T(P,N)= T_{serial} + T_{parallel} .$$ 

<p style="text-align:center;">
    <img src="./images/gustafson.png" width=600 alt="gustafson"/>
    <div style="text-align:center;">Based on:  <a href="https://link.springer.com/referenceworkentry/10.1007%2F978-0-387-09766-4_78">Encyclopedia of Parallel Computing - Gustafson’s Law</a></div>
</p>   

According to Gustafson's law, the effective speedup when running 𝑃 processors is given by:

$$S(P,N) = \alpha + P \times (1-\alpha)$$

Where,

$$\alpha =  \dfrac{T_{serial}}{T_{serial} + T_{parallel}}$$ 

Therefore, by keeping the execution time constant we can really **scale linearly** with the number of processors. The main idea is to increase the problem size $N$ and at the same time increase the number of processors $P$ such that the execution time remains constant.

<div class="alert alert-block alert-info">
    <p style="font-weight: bold;"><i class="fa fa-info-circle"></i>Derivation:</p>

Let's say a parallel algorithm takes time $T_{serial}$ for the serial part and  $T_{parallel}$ for the parallel part on $P$ processors:
$$T(P,N)= T_{serial} + T_{parallel} .$$ 
    
On one processor it would take:

$$T(1,N) = T_{serial} + P \times T_{parallel}.$$

So the speedup
$$S(P,N) = \frac{T(1,N)}{T(P,N)}= \frac{ T_{serial} + P \times T_{parallel}}{T_{serial} + T_{parallel}}$$ 
    
    
Let 
$$\alpha =  \dfrac{T_{serial}}{T_{serial} + T_{parallel}}$$ 
be the serial fraction of the execution time, which is a constant.
    
Therefore 
$$S(P,N) =\frac{ T_{serial}}{T_{serial} + T_{parallel}} +P \times  \frac{ T_{serial} + T_{parallel}-T_{serial}}{T_{serial} + T_{parallel}} = \alpha + P \times (1-\alpha) .$$

</div>    

In [ ]:
import matplotlib.pyplot as plt
import numpy as np


def gustafson_speedup(alpha, P):
    speedup = P - alpha * (P - 1)
    return speedup


def speedup_plot(speedup, alpha, P, title):
    plt.plot(P, speedup, label=r"$\alpha$={}".format(alpha))
    # Add title and axis names
    plt.title(title)
    plt.xlabel("Number of processors")
    plt.ylabel("Speedup")
    # Limits for the Y axis
    plt.xlim(0, num_processors + 10)
    plt.ylim(0, None)
    plt.grid(True)
    plt.legend()
    plt.tight_layout()


num_processors = 200
P = np.arange(1, num_processors, 1, dtype=int)
alpha = [0.1, 0.3, 0.5, 0.7]
for ai in alpha:
    speedup = gustafson_speedup(ai, P)
    speedup_plot(speedup, ai, P, "Gustafson's law")

Good weak scaling means we can solve $N$ times bigger problem in similar time by using $N$ times the processors used for the smaller problem.
<div class="alert alert-block alert-warning">

<div style="font-size: 150%;"></div>
<i class=\"fa fa-info-circle\"></i>
    
**Take home message:** speedup can be obtained by solving larger problems on larger computing resources.
</div>


<div class="alert alert-block alert-warning">

<div style="font-size: 150%;"><i class="fa fa-info-circle"></i>&nbsp;Important</div>

* Not every algorithm lends itself to parallelization
* Algorithm that lends itself to good parallelization may not be the best for single core performance
* In some problems there are other overheads such as communication between the parallel parts    
* Choosing the right algorithm is a critical decision

So the message is that choose algorithms with the end goal in mind.
</div>


## How does a computer program work, a (very) brief summary and recap

Before we delve into parallel processing using python we take a short look into how a computer program works.

When we run a program we start a **process** which is an instance of that program's execution. 

For example, if you run a serial python program 10 times **from the command line** you will have created 10 processes, even though they are running the same program.

**Note:** running cells in a notebook all happen in the same process.

A process and its associated resources are managed by the operating system.

Each process has a main **thread** of execution which on modern operating systems are allowed to spawn multiple threads which execute different parts of the program at the "same" time.


<p style="text-align:center;">
    <img src="./images/processes_threads.jpg" alt="Threads"/>
     <div style="text-align:center;">Source: <a href="https://subscription.packtpub.com/book/networking_and_servers/9781784396008/8/ch08lvl1sec69/multithreading-and-multiprocessing">Learning Python Network Programming - Multithreading and multiprocessing</a></div>
</p>

## Thread 
A Thread or a Thread of Execution (also called lightweight process) is defined in computer science as the smallest unit that can be scheduled in an operating system. Threads are usually contained in processes. source: https://www.python-course.eu/threads.php

Threads work in a shared memory paradigm and can cause issues in code that is not thread-safe. Due to this CPython implementation has a **Global Interpreter lock (GIL)** (see below).

Let's address this **GIL** elephant in the room.

## Global Interpretor lock (GIL)

In short, in Python only one thread controls the Python interpreter at a time.

**NOTE:** This is true for the most common implementation of Python, CPython. There exist other implementations which do not have a GIL.

As mentioned in https://wiki.python.org/moin/GlobalInterpreterLock :
> In CPython, the global interpreter lock, or GIL, is a mutex that protects access to Python objects, preventing multiple threads from executing Python bytecodes at once. The GIL prevents race conditions and ensures thread safety. In short, this mutex is necessary mainly because CPython's memory management is not thread-safe.

where Mutex (lock) is a programming flag used to grab and release an object.

This lock needs to be acquired and released in order for a thread to execute.

Therefore - as we learned in Section 1 - it prevents running code in a single Python interpreter on different cores in parallel! This is definitely a bad news for parallel processing.

However, the good news is that there are use cases where we can still leverage the multithreading and also there are libraries that can circumvent the GIL issue, e.g. [Numba](https://numba.readthedocs.io/en/stable/user/jit.html#nogil), [Pythran](https://pythran.readthedocs.io/en/latest/MANUAL.html#gil-interaction).

## IO bound vs CPU bound

In Python, we can achieve parallel speedup by either using multiple threads within the same process or spawning multiple processes. However, to decide the approach needed in a specific case, we need to find out if our problem is IO (input-output) or CPU bound.

**IO bound problem**: A program that spends most of the time waiting for fetching or writing data rather than computing.

**CPU bound problem**: A program that spends most of its time on computing.

IO speed is much slower than the speed at which a CPU can execute instructions (see below).

Example: one the newest Intel CPUs
Intel Core i9-10910: Max Frequency 5GHz -> 5 X $10^9$ instructions per second

**Recap: Latency Comparison Numbers (~2012)**

```
                                             Real time              On human scale
L1 cache reference ......................... 0.5 ns                 0.5 s
Execute typical instruction ................   1 ns                 1.0 s
L2 cache reference ........................... 7 ns                 7.0 s
Main memory reference ...................... 100 ns                ~1.3 minutes
Send 2K bytes over 1 Gbps network ....... 20,000 ns  =  20 µs       5.5 hours
SSD random read ........................ 150,000 ns  = 150 µs       1.7 days
Read 1 MB sequentially from memory ..... 250,000 ns  = 250 µs       2.9 days
Read 1 MB sequentially from SSD  ..... 1,000,000 ns  =   1 ms      11   days
Send packet CA->Netherlands->CA .... 150,000,000 ns  = 150 ms       4.8 years
```
Source: https://gist.github.com/jboner/2841832

## Concurrency

As see in the section above, IO/network bound problems can wait for significant time (compared to CPUs execution time) for data.
In a multi threaded application the GIL can be passed to another thread while one thread is still waiting for data. This can lead to a significant speed up in IO bound problems.

<p style="text-align:center;">
    <img src="./images/multithreading.png" alt="multithreading"/>
    <div style="text-align:center;">Source: <a href="https://cloudsek.com/how-do-you-achieve-concurrency-with-python-threads">https://cloudsek.com/how-do-you-achieve-concurrency-with-python-threads</a></div>
</p>

Therefore, we can profit from the concurrency of our problem. However, concurrent execution in this case gives an impression that every thread is executing at the same time by cleverly switching between the threads.

# Introduction to multithreading and multiprocessing

## `concurrent.futures` module

Python has a `concurrent.futures` module as a part of the Python Standard Library that provides a convenient way of doing multithreading (and multiprocessing which comes later).

To demonstrate the usage of this library let's start by creating a simple `sleeping()` function that sleeps for 1 second by default:

In [ ]:
%%time
import time


def sleeping(secs=1):
    """This function just sleeps for secs seconds
    This simulates an IO bound problem"""
    time.sleep(secs)
    print("I slept for {} seconds".format(secs))
    return secs


sleeping()

What happens if we call sleeping twice?

In [ ]:
%%time
import time


def sleeping(secs=1):
    """This function just sleeps for secs seconds
    This simulates an IO bound problem"""
    time.sleep(secs)
    print("I slept for {} seconds".format(secs))
    return secs


sleeping()
sleeping()

In this program what happens is the following:
1. The first sleep is executed
2. It blocks any further execution until the specified time is passed
3. Then the program proceeds with the second sleeping statement

As you can see it took 2 seconds of "wall time" to run this program.

Can we make the above program faster by employing multithreading? 

Answer: Yes!

### `ThreadPoolExecutor`

Here is how we could do that using the `ThreadPoolExecutor` from `concurrent.futures`:

In [ ]:
%%time
import time
from concurrent.futures import ThreadPoolExecutor


def sleeping(secs=1):
    """This function just sleeps for secs seconds
    This simulates an IO bound problem"""
    time.sleep(secs)
    print("I slept for {} seconds".format(secs))
    return secs


num_threads = 2
num_sleeps = 2

with ThreadPoolExecutor(max_workers=num_threads) as executor:
    for _ in range(num_sleeps):
        executor.submit(sleeping)

As you can see that the above program executes the `sleeping()` function two times, each on an independent thread and it took 1 second to run the program.  

The **context manager** (see line 17: `with`....) makes sure that the tasks are completed and the threads are cleaned up properly when we are done.

`executor.submit()` is responsible for scheduling the tasks.

### Exercise (10 Mins)
Play with the `num_threads` and `num_sleep` and see what happens.

### Future object
We can easily pass positional and keyword arguments to the `executor` using the `submit()` method:
- for a function `fn(*args, **kwargs)` we simply use `executor.submit(fn, *args, **kwargs)`.


`submit` returns a **futures object** which does not keep track of the task . We have to track it explicitly using the available methods such as:
- `running()` -  Returns True if the call is currently being executed and cannot be cancelled.

- `done()` - Returns True if the call was successfully cancelled or finished running.

- `result()` - Returns the value returned by the call. If the task is not completed this will wait.

- `exception()` - Returns the exception raised by the call.

For a complete list of the methods, please have a look at:
https://docs.python.org/3/library/concurrent.futures.html

For a list of future objects (or simply futures) it is practical to get them as they are completed. We can achieve this using `as_completed()` function:

In [ ]:
%%time
import time
from concurrent.futures import ThreadPoolExecutor, as_completed


def sleeping(secs):
    """This function just sleeps for secs seconds
    This simulates an IO bound problem"""
    time.sleep(secs)
    print("I slept for {} seconds".format(secs))
    return secs


num_threads = 4
secs = range(8, 0, -1)

with ThreadPoolExecutor(max_workers=num_threads) as executor:
    results = [executor.submit(sleeping, i) for i in secs]
    for tmp in as_completed(results):
        print(tmp.result())

The output was printed as it was completed!

In [ ]:
# Submit statement returns a futures object
print(results)

By iterating over the original futures we can get them in the order they appear in the list:

In [ ]:
for tmp in results:
    print(tmp.result())

### `executor.map()`
`concurrent.futures` also provides a `executor.map()` method that mimics Python `map()` and hence it can be applied on a list (or other iterator):

In [ ]:
%%time
import time
from concurrent.futures import ThreadPoolExecutor, as_completed


def sleeping(secs):
    """This function just sleeps for secs seconds
    This simulates an IO bound problem"""
    time.sleep(secs)
    print("I slept for {} seconds".format(secs))
    return secs


num_threads = 4
secs = range(8, 0, -1)

with ThreadPoolExecutor(max_workers=num_threads) as executor:
    results = executor.map(sleeping, secs)

for tmp in results:
    print(tmp)

**Note:** When using **executor.map()**, we do not get a future object but instead we directly get the data of the results returned by the function, in the order same as the inputs. Therefore, we do not have to use `as_completed()`.

### Error propagation

It is very important to keep in mind that error within one of the executions does not crash the entire program!


Error is raised when we try to access or do something with the result of the function.


The **futures object** however contains the error information.

In [ ]:
%%time
from concurrent.futures import ThreadPoolExecutor


def inverse(num):
    return 1 / num


num = range(8)

with ThreadPoolExecutor(max_workers=4) as executor:
    results = [executor.submit(inverse, i) for i in num]

    for tmp in results:
        print(tmp.exception())

As mentioned above, the actual error is thrown once we try to access the result.

In [ ]:
with ThreadPoolExecutor(max_workers=4) as executor:
    results = [executor.submit(inverse, i) for i in num]

    for tmp in results:
        print(tmp.result())

### Exercise: Downloading files [15 min]

A typical use case of multithreading is to download a bunch of files from the internet.

In this use case, threads generally have to wait for the network and IO and multithreading can offer substantial speedup.

In the following exercise we will download a bunch of images from the internet.

* Look at the code below and try to understand it from a high level perspective
* Complete the code by creating a threadpoolexecutor which takes num of threads as the arguments and download files independently on each thread using the function `downloader`
* Play with the number of threads and compare the execution times

In [ ]:
%%time
import os
import tempfile
from concurrent.futures import ThreadPoolExecutor

import requests


def downloader(args):
    """This function accepts a tuple (url, target_folder) and downloads the given file
    target folder."""

    url, target_folder = args
    # last string after last "/" is the filename, e.g.
    # url is
    # https://cdn.pixabay.com/photo/2015/07/13/21/54/gray-cat-843916__480.jpg
    # then the file name is gray-cat-843916__480.jpg:
    file_name = url.rsplit("/", 1)[1]

    target = os.path.join(target_folder, file_name)
    try:
        with requests.get(url, stream=True) as response:
            response.raise_for_status()  # raise exception for invalid URL
            with open(target, "wb") as fh:
                fh.write(response.raw.read())
        return file_name
    except requests.HTTPError as exception:
        print(exception)
        return None


# Reading in the urls of the images
# (source, cats.txt: https://github.com/amitupreti/image_downloader_multiprocessing_python)

with open("./data/cats.txt", "r") as fh:
    urls = [line.strip() for line in fh]

# restrict to 20 urls for testing:
urls = urls[:200]

target_folder = tempfile.mkdtemp()
print("download pictures to", target_folder)

args = [(url, target_folder) for url in urls]

names = list(map(downloader, args))

files_downloaded = list(filter(None, names))
print(len(files_downloaded))

# START TO COMPLETE HERE

# 1. Create a list of tuples (url, target_folder)
# 2. Use ThreadPoolExecutor to download the files
# 3. Print the number of downloaded files

#### Solution

In [ ]:
%%time
import os
import tempfile
from concurrent.futures import ThreadPoolExecutor

import requests


def downloader(args):
    """This function accepts a tuple (url, target_folder) and downloads the given file
    target folder."""

    url, target_folder = args
    # last string after last "/" is the filename, e.g.
    # url is
    # https://cdn.pixabay.com/photo/2015/07/13/21/54/gray-cat-843916__480.jpg
    # then the file name is gray-cat-843916__480.jpg:
    file_name = url.rsplit("/", 1)[1]

    target = os.path.join(target_folder, file_name)
    try:
        with requests.get(url, stream=True) as response:
            response.raise_for_status()  # raise exception for invalid URL
            with open(target, "wb") as fh:
                fh.write(response.raw.read())
        return file_name
    except requests.HTTPError as exception:
        print(exception)
        return None


# Reading in the urls of the images
# (source, cats.txt: https://github.com/amitupreti/image_downloader_multiprocessing_python)

with open("./data/cats.txt", "r") as fh:
    urls = [line.strip() for line in fh]

# restrict to n urls for testing:
urls = urls[:20]

target_folder = tempfile.mkdtemp()
print("download pictures to", target_folder)

num_threads = 2

args = [(url, target_folder) for url in urls]

with ThreadPoolExecutor(max_workers=num_threads) as executor:
    # list to trigger actual iteration over iterator object:
    names = list(executor.map(downloader, args))

files_downloaded = list(filter(None, names))

print("files where downloaded to", target_folder)
print("number of downloaded files:", len(files_downloaded))

### When does multithreading not help?
As already mentioned earlier multithreading will not be able to give us a speedup for CPU bound tasks. 

Let's demonstrate this with a very simple example that is compute intensive:


In [ ]:
def powern(list_numbers):
    """This function takes in a list of numbers,
    computes the power of 9999 of each and sums the results"""
    return sum([i ** 9999 for i in list_numbers])


list_numbers = [564982 + 100 * i for i in range(400)]

First we call `powern` a few times in a serial fashion:

In [ ]:
%%time
results = [powern(list_numbers) for _ in range(6)]

And now we try to improve speed using multiple threads:

In [ ]:
%%time
from concurrent.futures import ThreadPoolExecutor

num_workers = 3
with ThreadPoolExecutor(max_workers=num_workers) as executor:
    results = [executor.submit(powern, list_numbers) for _ in range(6)]

As you can see the `GIL` avoided speed-up.

In this case we have to run multiple Python interpreters which execute code in parallel. This is called multiprocessing.

### Multiprocessing with `ProcessPoolExecutor`

`concurrent.futures` provides a `ProcessPoolExecutor` that works same as the `ThreadPoolExecutor`.  
As the name suggests it is distributing work to multiple processes and not to multiple threads. This circumvents the limitations due to the `GIL`.

<div class="alert alert-block alert-warning">

<div style="font-size: 150%;"><i class="fa fa-info-circle"></i>&nbsp;Important: Mac and Windows users</div>

**Safe importing of main module:**
    
Make sure that the main module can be safely imported by a new Python interpreter without causing unintended side effects (such a starting a new process). 

For example, when running on Windows or macOS(), fence your code which triggers multi core execution as follows:
    
<div style="background: #ededed">
<code>if __name__ == "__main__":
    from concurrent.futures import ProcessPoolExecutor
    with ProcessPoolExecutor(max_workers=num_workers) as executor:
         ...
</code>
    </div>
<br> 
    
**Using multiprocessing in Jupyter notebooks:** 
    
Using multiprocessing in Jupyter notebooks on Mac and Window has some limitations and causes errors and crashes not seen on Linux. 
    
To prevent those we need to write the functions we want to run in parallel to a separate file, and then import the function, see also [here](https://stackoverflow.com/questions/47313732/jupyter-notebook-never-finishes-processing-using-multiprocessing-python-3/47374811#47374811).
</div>


In [ ]:
%%file powern.py

## Write to file and import later due to multiprocessing problems in Jupyter notebooks

from concurrent.futures import ProcessPoolExecutor

import numpy as np


def powern(list_numbers):
    """This function takes in a list of numbers, computes the power of 9999 of each and sums the results"""
    return sum([i ** 9999 for i in list_numbers])

In [ ]:
%%time

from concurrent.futures import ProcessPoolExecutor

from powern import powern

list_numbers = [564982 + 100 * i for i in range(400)]

num_workers = 3
with ProcessPoolExecutor(max_workers=num_workers) as executor:
    results = [executor.submit(powern, list_numbers) for _ in range(6)]

One difference to the `ThreadPoolExecutor` library is that the arguments that we pass to the functions that we `executor.submit` need to be **serializable/picklable**.



For more details please see [What can be pickled and unpickled?](https://docs.python.org/3/library/pickle.html#what-can-be-pickled-and-unpickled).

Other programming guidelines for using `multiprocessing` library correctly can be found here:
https://docs.python.org/3/library/multiprocessing.html#multiprocessing-programming

<div class="alert alert-block alert-warning">

<div style="font-size: 150%;"></div>
<i class=\"fa fa-info-circle\"></i>
    
<strong>Take home messages:</strong>
    
- Use Multithreading when the tasks need to wait for IO and network.
- Use Multiprocessing when the tasks are CPU bound.

</div>

<div class="alert alert-block alert-warning">

Some libraries with C code, such as NumPy, Scipy and Numba can release GIL and can be used with multithreading. 
    
Advantage of this is that threads are more "light-weight" than processes. 
    
Also, the communication between the processes has a higher overhead (objects need to be serialized etc).
</div>


<div class="alert alert-block alert-info">
    <p style="font-weight: bold;"><i class="fa fa-info-circle"></i>Side note, good to know, ...</p>

How does multiprocessing spawn the processes?

This depends on the operating system (So it is <strong>NOT</strong> platform independent)

source: https://docs.python.org/3/library/multiprocessing.html#the-spawn-and-forkserver-start-methods

* **spawn**
The parent process starts a fresh python interpreter process. The child process will only inherit those resources necessary to run the process object’s run() method. In particular, unnecessary file descriptors and handles from the parent process will not be inherited. Starting a process using this method is rather slow compared to using fork or forkserver.

    **Available on Unix and Windows. The default on Windows and macOS (As of Python 3.8).**

* **fork**
The parent process uses os.fork() to fork the Python interpreter. The child process, when it begins, is effectively identical to the parent process. All resources of the parent are inherited by the child process. Note that safely forking a multithreaded process is problematic.

    **Available on Unix only. The default on Unix.**

* **forkserver**
When the program starts and selects the forkserver start method, a server process is started. From then on, whenever a new process is needed, the parent process connects to the server and requests that it fork a new process. The fork server process is single threaded so it is safe for it to use os.fork(). No unnecessary resources are inherited.

    **Available on Unix platforms which support passing file descriptors over Unix pipes.**
</div>

### Oversubscription

**Demonstration:** Let's play with the `num_workers` and see what happens when you increase this number such that you use more workers than cores available, e.g. `num_workers` = 1,3,6,12 .

In [ ]:
%%time
num_workers = 4
with ProcessPoolExecutor(max_workers=num_workers) as executor:
    results = [executor.submit(powern, list_numbers) for _ in range(50)]

We can see that there is a maximum number of processes that we can use to get a speed up.  
Trying to use more resources than available is called **oversubscription**.  

<div class="alert alert-block alert-warning">

<div style="font-size: 150%;"><i class="fa fa-info-circle"></i>&nbsp;Important</div>

Oversubscription can slow down your program significantly! 
    
So if a program scales badly then make sure that you are not using more cores than available.
    
Oversubscription can also be hidden!!! For example when you are using a multicore-enabled `numpy` or `scipy` in your code <a href=https://stackoverflow.com/questions/6941459/is-it-possible-to-know-which-scipy-numpy-functions-run-on-multiple-cores>(read also here)</a>. 
    
Example: A `numpy` routine uses 4 cores and you try to speed it up by using `multiprocessing` with a pool of 4 workers. This results in an attempt to use 16 cores in total. In case your machine has less than 16 cores you will run into oversubscription.
    
If you would like to control this behavior of numpy, have a look at https://stackoverflow.com/questions/17053671/how-do-you-stop-numpy-from-multithreading       
    
</div>
</div>

For multiprocessing, the maximum number of processes, that can be used to get a speed up is equal to the number of virtual cores, can that be found using `os.cpu_count()`. 

**Warning**: There are caveats to this on HPC clusters that we will mention in **Section 6**. 

In [ ]:
import os

num_workers = os.cpu_count()
print(f"{num_workers=}")
# For Linux only
# print(len(os.sched_getaffinity(0)))

We can call `ProcessPoolExecutor` without specifying `max_workers` and in this case the number of CPUs is used:

<div class="alert alert-block alert-warning">

<div style="font-size: 150%;"><i class="fa fa-info-circle"></i>&nbsp;Note for Windows users:</div>

- Better use one worker less than available cores to avoid freezing of your machine.
- Multiprocessing scales badly on Windows since the start-up time for the workers is quite high.
    
</div>
</div>

## Traditional approch: `threading` and `multiprocessing`

Python Standard Library provides traditional libraries for doing multithreading and multiprocessing, `threading` and `multiprocessing` libraries, respectively. For writing concurrent code there is yet another library called `asyncio`. They are beyond the scope of the course. 

`concurrent.futures` relies on `threading` and `multiprocessing` and provides a unified, convenient, and simpler higher level API. However, once more control is needed `threading` and `multiprocessing` are the tools to use.


## Joblib

>Joblib is a set of tools to provide lightweight pipelining in Python. In particular:
> * transparent disk-caching of functions and lazy re-evaluation (memoize pattern)
> * easy simple parallel computing
> * Joblib is optimized to be fast and robust on large data in particular and has specific optimizations for numpy arrays.
>
> source: https://joblib.readthedocs.io/en/latest/

In this section, we will look at the parallel computing functionality of `joblib`.  
Joblib is perfect for parallelizing embarrassing parallel for loops.   
It is essentially a wrapper library which has different backends for parallel computing.  

By default Joblib uses the [loky](https://loky.readthedocs.io/en/stable/index.html) backend.
Loky boasts several advantages over `concurrent.futures` and `multiprocessing`:
* It claims to be more robust and interact better with third-party libraries (`multiprocessing` library is known to badly interact with some of these libraries).
* Along with pickle it also has an integration with the `cloudpickle` library which allows for serialization of several objects which are unpicklable. However, `cloudpickle` is slower.
* Deadlock free implementation: More robust behaviour w.r.t. deadlocks caused by crashing of workers. Although it is important to know that `concurrent.futures.ProcessPoolExecutor` as of Python 3.7+ is as robust as the executor from loky. However, loky would work well also for the older Python versions.

Further reading: https://loky.readthedocs.io/en/stable/index.html

**`joblib.Parallel`** - Helper class for readable parallel mapping

**`joblib.delayed`** - Decorator used to capture the arguments of a function

In [ ]:
%%time
from joblib import Parallel, delayed


def powern(list_numbers):
    """This function takes in a list of numbers, computes the power of 9999 of each and sums the results"""
    return sum([i ** 9999 for i in list_numbers])


def main():
    num_workers = 3
    with Parallel(n_jobs=num_workers) as parallel:
        results = parallel(delayed(powern)(list_numbers) for _ in range(6))


if __name__ == "__main__":
    main()

The backend can be changed by using `parallel_backend`:

In [ ]:
%%time
from joblib import parallel_backend

with parallel_backend("threading", n_jobs=num_workers):
    with Parallel(n_jobs=3) as parallel:
        results = parallel(delayed(powern)(list_numbers) for _ in range(6))

### Demonstration: Unpicklable objects with Joblib

In [ ]:
%%file lambda_func.py

square = lambda x: x ** 2

In [ ]:
%%time
from concurrent.futures import ProcessPoolExecutor
from multiprocessing import Pool

from joblib import Parallel, delayed

from lambda_func import square


def main():
    # with Pool(2) as executor:
    #    results = pool.map(square, [i for i in range(10)])
    with Parallel(n_jobs=2) as parallel:
        results = parallel(delayed(square)(i) for i in range(10))
    #with ProcessPoolExecutor(2) as executor:
    #    results = executor.map(square, [i for i in range(10)])


if __name__ == "__main__":
    main()

In the code example above, you can observe that passing a `lambda function` (unpicklable object) works with `joblib`.

Uncomment the lines using `ProcessPoolExecutor` and `Pool` (not discussed in this lecture) and see what happens?

As you can see Python allows us to parallelize the code on a PC with ease.

- If you start from scratch you should probably try `concurrent.futures` and `joblib`.  
- In case these libraries do not provide the functionality that you are looking for, you should use `threading` or `multiprocessing`, modules not covered in this course, as appropriate.

## Exercise - Compute Pi  [20 min]

Parallelize the $\pi$ approximation exercise using  `ThreadPoolExecutor` and `ProcessPoolExecutor` from `concurrent.futures`, and `joblib`.

1. What would you use?
2. Why?
3. Did you gain any speedup?


In [ ]:
pycat ../examples/pi.py

### Solution

In [ ]:
%%file approx_pi.py

from random import uniform


def approx_pi_mod(n_attempts):
    n_hits = 0

    for _ in range(n_attempts):
        x = uniform(-1.0, 1.0)
        y = uniform(-1.0, 1.0)
        if x ** 2 + y ** 2 <= 1.0:
            n_hits += 1
    return n_hits


In [ ]:
%%time
# Computing Pi
# Copy this code to a file and run on the command line

import time
from concurrent.futures import ProcessPoolExecutor, ThreadPoolExecutor

from joblib import Parallel, delayed

from approx_pi import approx_pi_mod


def main():
    num_points = 2_000_000
    num_workers = 4

    num_points_worker = [int(num_points / num_workers) for _ in range(num_workers)]

    num_points = sum(num_points_worker)

    with ProcessPoolExecutor(max_workers=num_workers) as executor:
        results = executor.map(approx_pi_mod, num_points_worker)

    # Solution using Joblib
    # with Parallel(n_jobs=num_workers) as parallel:
    #    results = parallel(delayed(approx_pi_mod)(i) for i in num_points_worker)

    print("The estimates value of Pi is: {}".format(sum(results) * 4 / num_points))


if __name__ == "__main__":
    main()

# Advanced topics in multiprocessing and multithreading

## Profiling of multithreaded programs
Profiling tools like **`cProfile`** lack support for multi-threaded programs.

An out of the box solution is <a href="https://github.com/sumerc/yappi"> **`yappi`** (Yet Another Python Profiler) </a>

We skip a demonstration of yappi here as it's output is hard to read and understand, especially compared to the profiling tools we demonstrated in section 2.

## Memory profile of multiprocessing programs
We can use `memory-profiler` at command line , i.e. `mprof`, together with `--multiprocess` and `--include-children` flags. The latter is especially needed if you use `subprocess` from the standard library and want to include the memory consumption of the spawned processes.

In [ ]:
%%file memory_benchmark.py
# Memory profiler for multiprocessing
# Chapter 2 memory profiler: mprof

import time
from concurrent.futures import ProcessPoolExecutor

import numpy as np


def outer_product(n):
    a = np.random.random(size=(n,))
    b = compute_b(a)


def compute_b(a):
    AA = np.outer(a, a)
    b = np.outer(a, a) @ a
    time.sleep(1)
    return b


def main():
    n = 10 * [100, 1_000, 10_000]

    num_workers = 2

    start_time = time.time()

    with ProcessPoolExecutor(max_workers=num_workers) as executor:
        results = executor.map(outer_product, n)

    end_time = time.time()

    print("Program took {} seconds".format(end_time - start_time))


if __name__ == "__main__":
    main()

In [ ]:
!mprof run --multiprocess --include-children memory_benchmark.py

In [ ]:
# create the plot
!mprof plot -o mprof.png

In [ ]:
# show plot in notebook

from IPython.display import Image

Image(filename="./mprof.png")

## Shared Memory and Multithreading

One has to be very careful of the so-called **race conditions** when using multithreading.

An example of this is as follows:

In [ ]:
%%time
from concurrent.futures import ThreadPoolExecutor

total = 0


def increment(n):
    """This function updates the global variable total by 1"""
    global total
    for i in range(n):
        total = total + 1


n = 10_000
m = 100

with ThreadPoolExecutor(max_workers=2) as executor:
    results = executor.map(increment, [n] * m)

print()
print(f"{total=}\t expected={n*m}\t ratio={round(total/(n*m),2)}")
print()

As you can see the final value of `total` is not as expected.

These generally occur when a variable is shared between the threads. The threads may read the same value of the variable and apply their respective operations. The threads will then write the updated value to the shared variable and the thread to do that last would overwrite the value written by the previous thread.



This is what you might think what happens:

| Thread 1| Thread 2|
|---------|---------|
| read total = 0 into register X      |         |
| increment register X to 1       |         |
| write register X, thus total = 1 | |
|         | read total = 1 into register Y       |
|         | increment register Y to 2        |
| | write register Y, thus total = 2 |

And this is what sometimes actually happens:


| Thread 1| Thread 2|
|---------|---------|
| read total = 0 into register X      |         |
| increment register X to 1       |         |
|         | read total = 0 into register Y       |
|         | increment register Y to 1        |
| write register X, thus total = 1 | |
| | write register Y, thus total = 1 |


### Optional exercise: 

Play with the `n` and `m` and see what happens! Try `n=10_000` and `m=100`; `n=1_000_000` and `m=1`;`n=1` and `m=10_000`?

In case the `total` is different than the `expected` value you encounter the **race condition**: the global variable `total` is shared between threads they is no guarantee that the value is properly synchronized between them.

## Shared memory and Multiprocessing

Let's run the previous example using multiprocessing.

<div class="alert alert-block alert-warning">

<div style="font-size: 150%;"><i class="fa fa-info-circle"></i>&nbsp;Important:</div>

In the examples in this section we use:

<div style="background: #ededed">
<code> mp_context=multiprocessing.get_context("fork")
</code>
    </div>
<br>
    
in the `ProcessPoolExecutor` calls.
    
This is because the features provided by the multiprocessing library to share data between processes do not work on Windows and MacOS, as advertised, with their default `spawn` context.
    
</div>
</div>



In [ ]:
%%file increment.py

import multiprocessing as mp
from concurrent.futures import ProcessPoolExecutor

total = 0


def increment(n):
    """This function updates the global variable total by 1"""
    global total
    for i in range(n):
        total = total + 1


def main():
    n = 10_000
    m = 100

    with ProcessPoolExecutor(
        max_workers=2, mp_context=mp.get_context("fork")
    ) as executor:
        results = executor.map(increment, [n] * m)

    print(f"{total=}\t expected={n*m}\t ratio={round(total/(n*m),2)}")


if __name__ == "__main__":
    main()

In [ ]:
!python increment.py

`total` is not incremented. Why?

As we just learned, when we use multiprocessing the variables are serialized and sent to child processes.
Avoiding serialization can be useful in some cases, improving the time to solution.  
We tried to achieve this using a global module variable, but things did not work as we expected.

Let's investigate this further by first just reading the value of a global variable, and checking its location in memory using the `id` function.

In [ ]:
%%file print_total.py

import multiprocessing as mp
from concurrent.futures import ProcessPoolExecutor

total = 6


def print_total_and_return_id(location="main"):
    global total
    print(total, location)
    return id(total)


def main():
    m = 6
    print_total_and_return_id()
    with ProcessPoolExecutor(
        max_workers=3, mp_context=mp.get_context("fork")
    ) as executor:
        results = executor.map(print_total_and_return_id, ["worker"] * m)
    print_total_and_return_id()

    print("Number of different locations in memory for total =", len(set(results)))
    print(f"{total=}")


if __name__ == "__main__":
    main()

In [ ]:
!python print_total.py

<div class="alert alert-block alert-info">
    <p style="font-weight: bold;"><i class="fa fa-info-circle"></i>id() function:</p>

Return the “identity” of an object. This is an integer which is guaranteed to be unique and constant for this object during its lifetime. Two objects with non-overlapping lifetimes may have the same id() value. CPython implementation detail: This is the address of the object in memory.
    
   source: https://docs.python.org/3/library/functions.html#id
    </div>

In multiprocessing, global module constants are seen by the child processes, and they really point to the same location in memory, so they are shared memory global constants.

Next we will try to update the value of `total` and check its memory location:

In [ ]:
%%file total_id.py

import multiprocessing as mp
from concurrent.futures import ProcessPoolExecutor

total = 0


def total_update_id(a):
    global total
    total = total + 1
    return id(total)


def main():
    m = 6

    with ProcessPoolExecutor(
        max_workers=3, mp_context=mp.get_context("fork")
    ) as executor:
        results = executor.map(total_update_id, [None] * m)

    print("Number of different locations in memory for total =", len(set(results)))
    print(f"{total=}")


if __name__ == "__main__":
    main()

In [ ]:
!python total_id.py

As we can see the global variables are not bounded to their location in memory once we try to update their value.

Using global variables and updating their value is a source of surprises.

In order to have shared variables we can use `multiprocessing.sharedctypes` or `multiprocessing.Manager` .

### multiprocessing.sharedctypes

If you want to have a shared variable in a multiprocessing setting you can use

- `multiprocessing.sharedctypes.Value()` or 
- `multiprocessing.sharedctypes.Array()`.

The first corresponds to a shared `ctypes` object (that we have learned about in Section 4), while the second to an array.

Let's use `multiprocessing.sharedctypes`:

In [ ]:
%%file total_id.py
import multiprocessing as mp
from concurrent.futures import ProcessPoolExecutor
from ctypes import c_int
from multiprocessing.sharedctypes import Value

total = Value(c_int, 0)


def total_update_id(a):
    global total
    total.value = total.value + 1
    return id(total)


def main():
    m = 6

    with ProcessPoolExecutor(
        max_workers=3, mp_context=mp.get_context("fork")
    ) as executor:
        results = executor.map(total_update_id, [None] * m)

    print("Number of different locations in memory for total =", len(set(results)))
    print(f"{total.value=}")


if __name__ == "__main__":
    main()

In [ ]:
!python total_id.py

So indeed, the same memory location is used for all processes.  
We can access the value with the attribute `.value`.

It is the time to check our initial example demonstrating race conditions:

In [ ]:
%%file increment.py

import multiprocessing as mp
import time
from concurrent.futures import ProcessPoolExecutor
from ctypes import c_int
from multiprocessing.sharedctypes import Value

total = Value(c_int, 0)


def increment():
    global total
    total.value = total.value + 1


def multi_increment(n):
    for i in range(n):
        increment()


def main():
    n = 10_000
    m = 100

    start = time.time()
    with ProcessPoolExecutor(
        max_workers=2, mp_context=mp.get_context("fork")
    ) as executor:
        results = executor.map(multi_increment, [n] * m)

    print(f"Time take = {time.time()-start}")
    print(f"{total.value=}\t expected={n*m}\t ratio={round(total.value/(n*m),2)}")


if __name__ == "__main__":
    main()

In [ ]:
!python increment.py

Even if `total` is shared, the total does not correspond to the expected value.
This is because of the race condition that we had seen while applying multithreading!

For operations that are not atomic (i.e., read/write) we have to lock the variable using `.get_lock()` method to avoid a race condition, but with a penalty on the execution time.

In [ ]:
%%file increment.py

import multiprocessing as mp
import time
from concurrent.futures import ProcessPoolExecutor
from ctypes import c_int
from multiprocessing.sharedctypes import Value

total = Value(c_int, 0)


def increment():
    global total
    with total.get_lock():
        total.value = total.value + 1


def multi_increment(n):
    for i in range(n):
        increment()


def main():
    n = 10_000
    m = 100

    start = time.time()
    with ProcessPoolExecutor(
        max_workers=2, mp_context=mp.get_context("fork")
    ) as executor:
        results = executor.map(multi_increment, [n] * m)

    print(f"Time take = {time.time()-start}")
    print(f"{total.value=}\t expected={n*m}\t ratio={round(total.value/(n*m),2)}")


if __name__ == "__main__":
    main()

In [ ]:
!python increment.py

By default, the `Value` are locked, but only for atomic operations. If you want to completely remove the locking, you can initialize the value with `total = Value(c_int, 0, lock=False)` (and of course also remove the `total.get_lock()` block). 

### multiprocessing.Manager

In case you need to share `list` or `dict`, you can use `multiprocessing.Manager`. 
> A manager object returned by Manager() controls a server process which holds Python objects and allows other processes to manipulate them using proxies.

This approach allows for higher flexibility, but it is slower.

In [ ]:
%%file managers.py

import multiprocessing as mp
from concurrent.futures import ProcessPoolExecutor
from multiprocessing import Manager


def print_shared(task_id, shared_dict):
    print(shared_dict[task_id])


def main():
    task_id = range(10)

    manager = Manager()
    shared_dict = manager.dict({el: el * 10 for el in task_id})

    with ProcessPoolExecutor(max_workers=3) as executor:
        results = executor.map(print_shared, task_id, [shared_dict] * len(task_id))


if __name__ == "__main__":
    main()

In [ ]:
!python managers.py

The manager itself has a lock `Lock()` that can be used to deal with operations that are not atomic.  
In practice we recommend you to avoid locking operations.

<div class="alert alert-block alert-info">
    <p style="font-weight: bold;"><i class="fa fa-info-circle"></i>Working with large data</p>
    
**Using Multiprocessing's shared memory functionality**

When working with large data, serialization while using multiprocessing can cause high computational and memory costs.

Multiprocessing introduced a module `multiprocessing.shared_memory` as of Python 3.8 which allows for shared memory across processors.

A template procedure to use this is as follows:
* Create a shared memory block of specified size
* Create a numpy array passing in the created buffer
* Copy data to this array
* Pass the name of the shared memory to the function where the data processing takes place
* Within the function recreate the numpy array based on the data in this shared memory without duplicating the data
* Process the data

see: https://mingze-gao.com/posts/python-shared-memory-in-multiprocessing/ for a working example
    
**Joblib's support of NumPy's memmap**
    
For working with large data, NumPy's `memmap` can be passed to joblib.Parallel.

Have a look at this example from the official documentation.

https://joblib.readthedocs.io/en/latest/auto_examples/parallel_memmap.html
    
</div>